# Solving for the Chebyshev-Based Spectral Derivative at Domain Endpoints

When solving for the Chebyshev-based derivative via detour to the Fourier basis, there is a major problem in the expanded, "splintered" expressions for $\frac{d^\nu}{dx^\nu} y(\theta)$: Factors in $x$ multiplying each $y{(\nu)} = \frac{d^\nu}{d\theta^\nu}$ have multiples of $\sqrt{1-x^2}$ in their denominators, making them undefined for $x = \pm 1$. We can solve for the endpoints using a L'Hôpital's rule, but the answers are difficult to derive, especially for higher derivatives.

I opened [Issue #1](https://github.com/pavelkomarov/spectral-derivatives/issues/1) to address the ugliness of this with a programmatic solution. An algorithm to derive the endpoint expressions turns out to be pretty challenging, not so easy to turn into compact code, and unfortunately relies on symbolic differentiation and simplification. As such, it has not been put in to the core library code. However, in case anyone should need to take a derivative beyond the $4^{th}$, this notebook implements a process to find the right expressions.

In [1]:
import sympy as sp
from collections import defaultdict, deque
from numpy.polynomial import Polynomial as poly

In [2]:
order = 11 # The order of derivative you want to work with

## Using $\theta$ Instead of $x$

$$ \frac{d}{dx} y(\theta) = \frac{d}{d\theta}y(\theta) \cdot \frac{d\theta}{dx}$$

And then subsequent derivatives entail a product rule, which is how terms quickly proliferate. These derivatives are pretty ugly, because $\frac{d\theta}{dx} = \frac{d}{dx}\cos^{-1}(x) = \frac{-1}{\sqrt{1-x^2}}$. However, due to the relationship $x = \cos(\theta)$, $\frac{-1}{\sqrt{1-x^2}} = \frac{-1}{\sin(\theta)}$. To avoid taking the limit of a quantity with multiple related variables, it's somewhat cleaner to put everything in terms of $\theta$ and then apply L'Hôpital's rule by taking derivatives w.r.t. $\theta$ rather than $x$. The two are equivalent; we end up with powers of $\sin(\theta)$ in the denominator, which is also stubbornly 0 at the endpoints ($\theta = 0$ and $\pi$).

## Finding the Denominator

If we carry through with L'Hôpital's rule in variable $\theta$, we see that after each differentation a $\sin(\theta)$ cancels with all the terms in the numerator, just like how when we do it in $x$ a $\sqrt{1-x^2}$ cancels, and we can thereby, over the course of $\nu$ applications of the rule, eliminate the troublesome $\sin(\theta)$s in the denominator.

The denominator starts out $\sin^{2\nu-1}(\theta)$ and gets changed simply as in the following loop:

In [3]:
th = sp.symbols('th')
denom = sp.sin(th)**(2*order - 1)
for i in range(order-1):
	denom = sp.diff(denom, th)
	denom = sp.cancel(denom/sp.sin(th))
denom = sp.diff(denom, th)
print(denom)

-151242416325*sin(th)**10*cos(th) + 2268636244875*sin(th)**8*cos(th)**3 - 6352181485650*sin(th)**6*cos(th)**5 + 4537272489750*sin(th)**4*cos(th)**7 - 756212081625*sin(th)**2*cos(th)**9 + 13749310575*cos(th)**11


We can now evaluate at the endpoints and get nonzero values. For even $\nu$ the two endpoints have the same denominator; for odd $\nu$ they are $\pm$ each other.

In [4]:
D_0 = denom.subs(th, 0)
D_pi = denom.subs(th, sp.pi)
print(D_0, D_pi)

13749310575 -13749310575


## Finding the Numerator Terms

We can begin with the pyramid of terms built up for $\frac{d^\nu}{dx^\nu} y(\theta)$ as evaluated in the package.

In [5]:
# Calculate the polynomials in x necessary for transforming back to the Chebyshev domain
numers = deque([poly([-1])]) # just -1 to start, at order 1
denom = poly([1, 0, -1]) # 1 - x^2
for nu in range(2, order + 1): # initialization takes care of order 1, so iterate from order 2
	q = 0
	for mu in range(1, nu): # Terms come from the previous derivative, so there are nu - 1 of them here.
		p = numers.popleft() # c = nu - mu/2
		numers.append(denom * p.deriv() + (nu - mu/2 - 1) * poly([0, 2]) * p - q)
		q = p
	numers.append(-q)

These are functions in $x$, but we can substitute to easily make them functions of $\cos(\theta)$, and then we can multipy by appropriate powers of $\sin(\theta)$ to put them all in the numerator of a single fraction.

In [6]:
exprs = deque()
for mu,p in enumerate(numers):
	expr = 0
	for r,c in enumerate(p.coef):
		expr += int(c) * sp.cos(th)**r
	expr *= sp.sin(th)**mu
	exprs.append(expr)

In [7]:
print(exprs)

deque([-3628800*cos(th)**10 - 81648000*cos(th)**8 - 285768000*cos(th)**6 - 238140000*cos(th)**4 - 44651250*cos(th)**2 - 893025, (10628640*cos(th)**9 + 144666720*cos(th)**7 + 324389340*cos(th)**5 + 161191800*cos(th)**3 + 13852575*cos(th))*sin(th), (-12753576*cos(th)**8 - 107213436*cos(th)**6 - 149044896*cos(th)**4 - 40065696*cos(th)**2 - 1057221)*sin(th)**2, (8409500*cos(th)**7 + 43600260*cos(th)**5 + 35543805*cos(th)**3 + 4338235*cos(th))*sin(th)**3, (-3416930*cos(th)**6 - 10681275*cos(th)**4 - 4647885*cos(th)**2 - 172810)*sin(th)**4, (902055*cos(th)**5 + 1619310*cos(th)**3 + 316470*cos(th))*sin(th)**5, (-157773*cos(th)**4 - 148764*cos(th)**2 - 8778)*sin(th)**6, (18150*cos(th)**3 + 7590*cos(th))*sin(th)**7, (-1320*cos(th)**2 - 165)*sin(th)**8, 55*sin(th)**9*cos(th), -sin(th)**10])


Now we note that these are multiplying subsequent orders of $y^{(\mu)}(\theta)$, so the full numerator looks like:

$$a(\theta)y'(\theta) + b(\theta)y''(\theta) + ... z(\theta)y^{(\nu)}(\theta)$$

This situation will mean product rules as we take derivatives. Mildly gross, but like the pyramid scheme from before, each $y^{(\mu)}(\theta)$ depends only on two terms from the previous expression, so we can evaluate subsequent numerators actually pretty easily.

For example:

$$ay' + by'' + cy''' \rightarrow a'y' + (a + b')y'' + (b + c')y''' + cy^{IV}$$

We can line these up to see an obvious relationship of each new term to the one above and the one up-left-diagonal:

$$\begin{array}{c c c c}
a & b & c \\
a' & a+b' & b+c' & c\\
\end{array}$$

Then before the next iteration of this process we divide by a $\sin(\theta)$ to cancel one from the denominator.

We need to do this $\nu$ times to get all the way up to the $(2\nu)^{th}$ derivative before the denominator stops being 0. However, a beautiful thing happens to the numerator at the second-to-last of these additional iterations: All the terms involving even derivatives of $y(\theta)$ cancel with themselves, and all the terms involving odd derivatives are multiplied by only *constant* factors. It's like a Law of Nature. This is necessary to fit the $\frac{0}{0}$ indeterminate form one final time, because we've canceled so many $\sin(\theta)$ that there are no longer any left to multiply (and thereby zero out) the even-derivative terms (which would otherwise be nonzero) in the numerator. Then the last derivative entails no product rule, only shifts the constant coefficients of the odd derivatives to the next-highest even derivatives. So we can really stop the iteration one short, since all we need are those constants.

In [8]:
for nu in range(order + 1, 2*order): # iterate order-1 more times to reach the constants
	q = 0
	for mu in range(1, nu): # Terms come from the previous derivative, so there are nu - 1 of them here.
		p = exprs.popleft()
		term = q + sp.diff(p, th)
		exprs.append(sp.cancel(term/sp.sin(th)))
		q = p
	exprs.append(sp.cancel(q/sp.sin(th)))
exprs = [expr.rewrite(sp.exp).simplify() for expr in exprs] # rewriting as exponentials before simplification helps sympy
print(exprs)

[-13168189440000, 0, -20407635072000, 0, -8689315795776, 0, -1593719752240, 0, -151847872396, 0, -8261931405, 0, -268880381, 0, -5293970, 0, -61446, 0, -385, 0, -1]


Sympy can struggle if we're not careful here. See https://stackoverflow.com/questions/79404210/how-to-cancel-trigonometric-expressions-in-sympy

In [9]:
C = [int(exprs[i]) for i in range(0, 2*order-1, 2)] # constants
print(C)

[-13168189440000, -20407635072000, -8689315795776, -1593719752240, -151847872396, -8261931405, -268880381, -5293970, -61446, -385, -1]


We can pick out that $y''(\theta)$ needs to be multiplied by the first value, $y^{IV}(\theta)$ by the second, and so on.

## Putting it All Together

If we use the DCT$^{-1}$ to reconstruct the first and last points, the answer will be:

$$
\begin{align} \frac{1}{D_0 M} \Big((... - C_3 N^6 + C_2 N^4 - C_1 N^2) Y_N + 2 \sum_{k=1}^{N-1} (... - C_3 k^6 + C_2 k^4 - C_1 k^2) Y_k \Big) & \text{ at } x=1, \theta=0 \\ \frac{1}{D_\pi M} \Big((... - C_3 N^6 + C_2 N^4 - C_1 N^2)(-1)^N Y_N + 2 \sum_{k=1}^{N-1} (... - C_3 k^6 + C_2 k^4 - C_1 k^2) (-1)^k Y_k \Big) & \text{ at } x=-1, \theta=\pi \end{align}
$$

Where the alternating plus and minus come from the fact the $2^{nd}$ derivative contains `-cos`, the $4^{th}$ `cos`, the $6^{th}$ `-cos` again, and so on.

Let's make a code string that implements that, for the `order` given.

In [10]:
Ns = "".join((" + " if i % 2 else " - ") + f"{C[i]}" + (f"*N**{i*2}" if i > 0 else "") for i in range(len(C)-1, -1, -1))
ks = "".join((" + " if i % 2 else " - ") + f"{C[i]}" + f"*k**{(i+1)*2}" for i in range(len(C)-1, -1, -1))
d = {"- -":"+ ", "+ -":"- ", " + 1*":"", " 1*":" "}
for k,v in d.items(): Ns = Ns.replace(k, v)
for k,v in d.items(): ks = ks.replace(k, v)

print(f"dy_n[first] = np.sum(({ks})[s] * Y_k[middle], axis=axis)/({D_0}*N) + N*({Ns})/{2*D_0} * Y_k[last]")
print(f"dy_n[last] = np.sum((({ks})*np.power(-1, k))[s] * Y_k[middle], axis=axis)/({D_pi}*N) + (N*({Ns})*(-1)**N)/{2*D_pi} * Y_k[last]")

dy_n[first] = np.sum((k**22 - 385*k**20 + 61446*k**18 - 5293970*k**16 + 268880381*k**14 - 8261931405*k**12 + 151847872396*k**10 - 1593719752240*k**8 + 8689315795776*k**6 - 20407635072000*k**4 + 13168189440000*k**2)[s] * Y_k[middle], axis=axis)/(13749310575*N) + N*(N**20 - 385*N**18 + 61446*N**16 - 5293970*N**14 + 268880381*N**12 - 8261931405*N**10 + 151847872396*N**8 - 1593719752240*N**6 + 8689315795776*N**4 - 20407635072000*N**2 + 13168189440000)/27498621150 * Y_k[last]
dy_n[last] = np.sum(((k**22 - 385*k**20 + 61446*k**18 - 5293970*k**16 + 268880381*k**14 - 8261931405*k**12 + 151847872396*k**10 - 1593719752240*k**8 + 8689315795776*k**6 - 20407635072000*k**4 + 13168189440000*k**2)*np.power(-1, k))[s] * Y_k[middle], axis=axis)/(-13749310575*N) + (N*(N**20 - 385*N**18 + 61446*N**16 - 5293970*N**14 + 268880381*N**12 - 8261931405*N**10 + 151847872396*N**8 - 1593719752240*N**6 + 8689315795776*N**4 - 20407635072000*N**2 + 13168189440000)*(-1)**N)/-27498621150 * Y_k[last]




## The Nightmare of Splintering in Higher Dimensions

When reducing Chebyshev derivatives to Fourier ones, converting back to the Chebyshev domain is already pretty knotty in one dimension, because we have to keep all inverse transforms in $\theta$ up to order $\nu$ to compute the derivative of order $\nu$ in $x$, but it turns out to be much worse in the simultaneous-multidimensional derivatives case, due to interactions. I'll demonstrate with the simplest possible case, $2^{nd}$ order in one dimension and $1^{st}$ order in another:

\begin{equation}
\frac{\partial^3}{\partial x_1^2 \partial x_2} y(\theta_1, \theta_2) = \frac{\partial}{\partial x_2} \Big[ \frac{\partial^2}{\partial x_1^2} y(\theta_1, \theta_2) \Big]
\end{equation}

Let's break this up and apply the [multivariable chain rule](https://math.libretexts.org/Bookshelves/Calculus/Calculus_(OpenStax)/14%3A_Differentiation_of_Functions_of_Several_Variables/14.05%3A_The_Chain_Rule_for_Multivariable_Functions) and product rule to just evaluate that inner portion first:

\begin{align*}
\frac{\partial^2}{\partial x_1^2} y(\theta_1, \theta_2) &= \frac{\partial}{\partial x_1} \Big( \frac{\partial}{\partial \theta_1} y \cdot \frac{d\theta_1}{dx_1} + \frac{\partial}{\partial \theta_2} y \cdot \overset{\text{\normalsize 0}}{\cancel{\frac{d\theta_2}{dx_1}}} \Big) & \text{multivariable chain rule}\\
&= \frac{\partial}{\partial \theta_1} y \cdot \frac{d^2 \theta_1}{dx_1^2} + \frac{\partial}{\partial x_1}\Big( \frac{\partial}{\partial \theta_1} y \Big) \cdot \frac{d\theta_1}{d x_1} & \text{product rule}\\
&= \frac{\partial}{\partial \theta_1} y \cdot \frac{d^2 \theta_1}{dx_1^2} + \Big( \frac{\partial^2}{\partial \theta_1^2} y \cdot \frac{d \theta_1}{dx_1} + \frac{\partial^2}{\partial \theta_2 \partial \theta_1} y \overset{\text{\normalsize 0}}{\cancel{\frac{d\theta_2}{dx_1}}} \Big) \cdot \frac{d\theta_1}{d x_1} & \text{multivariable chain rule}\\
&= \frac{\partial}{\partial \theta_1} y \cdot \frac{d^2 \theta_1}{dx_1^2} + \frac{\partial^2}{\partial \theta_1^2} y \cdot \Big( \frac{d \theta_1}{dx_1} \Big)^2
\end{align*}

Now using this to evaluate the minimal example:

\begin{align*}
\frac{\partial^3}{\partial x_1^2 \partial x_2} y(\theta_1, \theta_2) =& \frac{\partial}{\partial x_2} \Big[ \frac{\partial}{\partial \theta_1} y \cdot \frac{d^2 \theta_1}{dx_1^2} + \frac{\partial^2}{\partial \theta_1^2} y \cdot \Big( \frac{d \theta_1}{dx_1} \Big)^2 \Big]\\
=& \frac{\partial}{\partial \theta_1} y \cdot \overset{\text{\normalsize 0}}{\cancel{\frac{\partial}{\partial x_2} \frac{d^2 \theta_1}{dx_1^2}}} + \Big( \frac{\partial^2}{\partial \theta_1^2} y \cdot \overset{\text{\normalsize 0}}{\cancel{\frac{d\theta_1}{dx_2}}} + \frac{\partial^2}{\partial \theta_1 \partial \theta_2} y \cdot \frac{d \theta_2}{dx_2} \Big) \cdot \frac{d^2\theta_1}{dx_1^2} \\
&+ \frac{\partial^2}{\partial \theta_1^2} y \overset{\text{\normalsize 0}}{\cancel{\frac{\partial}{\partial x_2} \Big( \frac{d\theta_1}{dx_1} \Big)^2}} + \Big( \frac{\partial^3}{\partial \theta_1^3} y \cdot \overset{\text{\normalsize 0}}{\cancel{\frac{d\theta_1}{dx_2}}} + \frac{\partial^3}{\partial \theta_1^2 \partial \theta_2} y \cdot \frac{d \theta_2}{dx_2} \Big) \cdot \Big(\frac{d\theta_1}{dx_1}\Big)^2 \\
=& \frac{\partial^2}{\partial \theta_1 \partial \theta_2} y \cdot \frac{d \theta_2}{dx_2} \cdot \frac{d^2\theta_1}{dx_1^2} + \frac{\partial^3}{\partial \theta_1^2 \partial \theta_2} y \cdot \frac{d \theta_2}{dx_2} \cdot \Big(\frac{d\theta_1}{dx_1}\Big)^2
\end{align*}

The double derivative in $x_1$ has *splintered* the expression into two, and then the single derivative in $x_2$ has interacted with *both those terms.

All the terms that involve derivatives of a $\theta$ w.r.t. an $x$ are ultimately just functions of $x$. In fact, $\frac{d\theta_2}{dx_2}$ is just our old friend $\frac{-1}{\sqrt{1-x_2^2}}$, and you can pick out $\big( \frac{d\theta_1}{dx_1} \big)^2$ and $\frac{d^2\theta_1}{dx_1^2}$ in the $\frac{d^2}{dx^2} y(\theta)$ equation. Together they are *a Cartesian product* of the the 1D case "pyramid"!

Meanwhile, the $\partial y$ terms have different orders, which means that to find them we need to multiply $Y$, the all-dimensions transform of $y$, by *different* orders of $jk$. If we do this carefully, the best-case scenario is that we incur [the same amount of work](https://github.com/pavelkomarov/spectral-derivatives/issues/2) as the in-series case, but it takes some extra bookkeeping and data copying.

Even worse, at the *edges* of the domain we still need to use L'Hôpital's rule on an analytic expression to evaluate the limits of\vspace{-2mm}

$$\frac{\partial^{\sum_i\nu_i}}{\partial x_1^{\nu_1} ... \partial x_D^{\nu_D}} y(\theta_1, ... \theta_D)$$

This is made more challenging by the fact our analytic reconstruction expression is based on the DCT-I$^{-1}$ or DCT-II$^{-1}$, which have terms outside the central sum, so as we substitute the DCT in to the DCT, we get ever more terms ($3^D$ of them for the DCT-I$^{-1}$ and $2^D$ of them for the DCT-II$^{-1}$), e.g.~in 2D the DCT-I$^{-1}$ is:\vspace{-2mm}

\begin{align*}
y(\theta_1, \theta_2) =& \frac{1}{M^2} \Big[ Y_{00} + Y_{N0}\cos(N\theta_1) + Y_{0N}\cos(N\theta_2) + Y_{NN}\cos(N\theta_1)\cos(N\theta_2)\\
&+ 2\sum_{k_1 = 1}^{N-1} Y_{k_1 0} \cos(k_1 \theta_1) + 2\sum_{k_2 = 1}^{N-1} Y_{0 k_2} \cos(k_2 \theta_2) + 2\cos(N\theta_2) \sum_{k_1 = 1}^{N-1} Y_{k_1 N} \cos(k_1 \theta_1)\\
&+ 2\cos(N\theta_1) \sum_{k_2 = 1}^{N-1} Y_{N k_2} \cos(k_2 \theta_2) + 4 \sum_{k_1 = 1}^{N-1} \sum_{k_2 = 1}^{N-1} Y_{k_1 k_2} \cos(k_1 \theta_1) \cos(k_2 \theta_2)\Big]
\end{align*}

We could generalize the original conception of the Chebyshev cosine series, which has a single sum with $a_0 = \frac{Y_0}{M}, a_k = \frac{2 Y_k}{M} \text{ for } k \in [1, N-1], a_N = \frac{Y_N}{M}$, to get a $y(\vec{\theta})$ with only a single term, but this involves still more extra bookkeeping.

Most gnarly, we then still have to take limits as different *combinations* of dimensions reach the edges, which becomes a combinatorial nightmare. This was already hard enough in 1D!

So although `numpy` does provide the `fftn` function for transforming in multiple dimensions at once, and `scipy` provides similar `dctn` and `dstn` functions, they wouldn't confer a computational-complexity benefit and would require the math and code to get massively more complicated, so I have chosen not to use them.